In [10]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from dtreeviz.trees import *
import sklearn

def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

traindf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory=False)
testdf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/test.csv',low_memory= False)

traindf['Train'] = True
testdf['Train'] = False

togeth = pd.concat([traindf,testdf])


togeth.drop(columns = 'Transported',inplace = True)


spending = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
togeth['Spending'] = togeth[spending].sum(axis = 1)


In [11]:
togeth.Spending.isna().sum()

0

CANT ASSERT CRYO FOR THOSE WITH NA IN SPENDING

In [12]:
def split_group_number(df):
    df['Group'] = df['PassengerId'].apply(lambda x: x.split("_")[0] if pd.notna(x) else pd.NA)
    df['GroupNumber'] = df['PassengerId'].apply(lambda x: x.split("_")[1] if pd.notna(x) else pd.NA)
    return df
togeth = split_group_number(togeth)

def split_names(df):
    # Create new columns for first name and last name
    df['FirstName'] = df['Name'].apply(lambda x: x.split()[0] if pd.notna(x) else pd.NA)
    df['LastName'] = df['Name'].apply(lambda x: x.split()[1] if pd.notna(x) else pd.NA)
    return df
togeth = split_names(togeth)


def split_cabin(df):
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split("/" )[0] if pd.notna(x) else pd.NA)
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split("/")[2] if pd.notna(x) else pd.NA)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split("/")[1]) if pd.notna(x) else pd.NA)
    return df
togeth = split_cabin(togeth)


In [13]:
def missing():
    return 12970 - len(togeth.dropna())
missing()

3083

Traindf 8693 - 6606 \
Testdf 4277 - 3281 \
Tot = 12970 - 9887


In [14]:
togeth.isna().sum()

PassengerId       0
HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Name            294
Train             0
Spending          0
Group             0
GroupNumber       0
FirstName       294
LastName        294
CabinDeck       299
CabinSide       299
CabinNum        299
dtype: int64

**CyroSleep**

if earth and cabindeck f then not cryosleep 99.3% 50ish \
 most in F and from Earth arent CryoSleep anyways

In [5]:
togeth.loc[(togeth.HomePlanet == 'Earth') & (togeth.CabinDeck == 'F'),'CryoSleep'] = togeth.loc[(togeth.HomePlanet == 'Earth') & (togeth.CabinDeck == 'F'),'CryoSleep'].fillna(False)

0 spending older 13 and cabindeck F then cryo True 99% added 0 \
most in f arent cryo anyways 

In [7]:
togeth.loc[(togeth.Age > 13) & (togeth.CabinDeck == 'F') & (togeth.Spending == 0),'CryoSleep'] = togeth.loc[(togeth.Age > 13) & (togeth.CabinDeck == 'F') & (togeth.Spending == 0),'CryoSleep'].fillna(True)

0 spending older 13 and cabindeck G then Cryo True 99% 30ish \
most not spending are in cryo anyways

In [8]:
togeth.loc[(togeth.Age > 13) & (togeth.CabinDeck == 'G') & (togeth.Spending == 0),'CryoSleep'] = togeth.loc[(togeth.Age > 13) & (togeth.CabinDeck == 'G') & (togeth.Spending == 0),'CryoSleep'].fillna(True)

Mars, D, spending = 0 then Cryo True 100% 20ish

In [9]:
togeth.loc[(togeth.HomePlanet == 'Mars')  & (togeth.Spending == 0),'CryoSleep'] = togeth.loc[(togeth.HomePlanet == 'Mars')  & (togeth.Spending == 0),'CryoSleep'].fillna(True)

In [10]:
togeth.loc[togeth.CabinDeck == 'T', 'CryoSleep'] = togeth.loc[togeth.CabinDeck == 'T', 'CryoSleep'].fillna(False)


In [11]:
missing()

2983

**Cabin**

**CabinDeck**

**CabinSide**

100% share Cabinside with group 230ish

In [12]:
def fillna_cabinside(df):
    groups = df.groupby('Group')
    for name, group in groups:
        group_side = group['CabinSide'].dropna().unique()
        if len(group_side) == 1:
            side_value = group_side[0]
            df.loc[group.index, 'CabinSide'] = side_value
    return df

# Call the function to fill NaN values in 'CabinSide'
togeth = fillna_cabinside(togeth)


**Destination**

In [13]:
traindf

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Train
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,True
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,True
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,True
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,True
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,True
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,True


**Spending**

if cyro then no spending 100% (400ish)

In [14]:
togeth.loc[togeth.CryoSleep == True, spending] = togeth.loc[togeth.CryoSleep == True, spending].fillna(0)


if under 13 then no spending 100 % (80ish)

In [15]:
togeth.loc[togeth.Age < 13, spending] = togeth.loc[togeth.Age < 13, spending].fillna(0)

**HomePlanet**

If you're in cabin A,B,C then youre from Europa 100% 40ish

In [16]:
togeth.loc[togeth['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'] = togeth.loc[togeth['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'].fillna('Europa')

gives same homplanet as someone else in your group 100% true 60ish

In [17]:
def fillna_homeplanet(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            group_filter = (df['Group'] == row['Group']) & ~df['HomePlanet'].isna()
            if group_filter.any():
                fill_value = df[group_filter]['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = fill_value
    return df
togeth = fillna_homeplanet(togeth)


fills homeplanet by matching lastname 100%

In [18]:
def fillna_homeplanet_with_lastname(df):
    homeplanet_mapping = df[df['HomePlanet'].notna()].groupby('LastName')['HomePlanet'].first().to_dict()
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']) and row['LastName'] in homeplanet_mapping:
            df.at[index, 'HomePlanet'] = homeplanet_mapping[row['LastName']]
    return df
togeth = fillna_homeplanet_with_lastname(togeth)


if youre in cabindeck G then from earth 100% 90ish

In [19]:
togeth.loc[togeth.CabinDeck == 'G','HomePlanet'] = togeth.loc[togeth.CabinDeck == 'G','HomePlanet'].fillna('Earth')

**VIP**

In [20]:
togeth.VIP.isna().sum()

296

Age < 18 then vip False 100% 50

In [21]:
togeth.loc[togeth.Age < 18,'VIP'] = togeth.loc[togeth.Age < 18, 'VIP'].fillna(False)

Age < 25 and Europa then VIP False 100% 5

In [22]:
togeth.loc[(togeth.HomePlanet == 'Europa') & (togeth.Age < 25),'VIP'] = togeth.loc[(togeth.HomePlanet == 'Europa') & (togeth.Age < 25),'VIP'].fillna(False)

HomePlanet Earth then not VIP 100% 100

In [23]:
togeth.loc[togeth.HomePlanet == 'Earth','VIP'] = togeth.loc[togeth.HomePlanet == 'Earth','VIP'].fillna(False)

Mars and CryoSleep True then not VIP 100% 20ish

In [24]:
togeth.loc[(togeth.HomePlanet == 'Mars') & (togeth.CryoSleep == True),'VIP'] = togeth.loc[(togeth.HomePlanet == 'Mars') & (togeth.CryoSleep == True),'VIP'].fillna(False)

CabinDeck G then not VIP 100%

In [25]:
togeth.loc[togeth.CabinDeck == 'G','VIP'] = togeth.loc[togeth.CabinDeck == 'G','VIP'].fillna(False)

Spending == 0 and not Cryo is super rare for VIP

In [26]:
togeth.loc[(togeth.Spending == 0)& (togeth.CryoSleep == False),'VIP'] = togeth.loc[(togeth.Spending == 0)& (togeth.CryoSleep == False),'VIP'].fillna(False)

In [27]:
missing()

2080

**Destination**

In [28]:
togeth.loc[(togeth.Age > 18)&(togeth.CryoSleep == False) & (togeth.Spending <400),'Destination']  = togeth.loc[(togeth.Age > 18)&(togeth.CryoSleep == False) & (togeth.Spending <400),'Destination'].fillna('TRAPPIST-1e')

In [29]:
togeth.isna().sum()

PassengerId       0
HomePlanet        9
CryoSleep       186
Cabin           299
Destination     268
Age             270
VIP              93
RoomService     162
FoodCourt       171
ShoppingMall    161
Spa             166
VRDeck          149
Name            294
Train             0
Spending          0
Group             0
GroupNumber       0
FirstName       294
LastName        294
CabinDeck       299
CabinSide        39
CabinNum        299
dtype: int64

In [30]:
traindf

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Train
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,True
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,True
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,True
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,True
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,True
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,True


In [32]:
newtrain = togeth.iloc[:8692].copy()
newtest = togeth.iloc[8693:]

In [33]:
len(newtest)

4277

In [34]:
newtrain['Transported'] = traindf['Transported']

In [35]:
newtrain.to_csv('newtrainer.csv',index=False)
newtest.to_csv('newtester.csv',index = False)

In [36]:
len(newtrain.dropna())

7309